In [ ]:
import sys

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
sys.path.insert(1, "/Users/helium/Documents/git/eeroqlab/zeroheliumkit/")

from zeroheliumkit import Structure, Rectangle, Square, Ring, Circle
from zeroheliumkit.src.settings import GRAY, BLUE, YELLOW2
from zeroheliumkit.src.plotting import ColorHandler

In [ ]:
save_dir = "dump/"
config_dir = "config/"

### **Creating geometry**

In [ ]:
device = Structure()
device.add_layer("wafer", Square(20))
device.add_layer("gnd", Square(20))
device.cut_polygon("gnd", Rectangle(3, 9))
device.cut_polygon("gnd", Rectangle(9, 3))
device.cut_polygon("gnd", Rectangle(9, 3, (0,5)))
device.add_layer("bottom", Circle(3))
device.add_polygon("bottom", Ring(3.5, 8))
device.add_layer("etch", Square(20))
device.cut_polygon("etch", device.gnd)

device.remove_holes_from_polygons("bottom")
device.remove_holes_from_polygons("gnd")

device.colors = ColorHandler({
    "wafer": GRAY,
    "bottom": YELLOW2,
    "gnd": (BLUE, 0.85)
})
device.quickplot(show_idx=True)

### **Creating Mesh**

In [ ]:
from zeroheliumkit.fem import gmshLayer_info, physSurface_info, GMSHmaker

In [ ]:
# extrude parameters
d_wafer     = 10
d_metal1    = 0.2
d_metal2    = 0.1
d_diel      = 1.5
d_vac       = 20
d_He        = d_diel + d_metal2

In [ ]:
""" 
extrude_config: 
key - gmsh layer name
item - gmshLayer_info dict: 
       (geometry layer, thickness, z0, Physical Vol Name, CUT gmsh layer=None, forConstruction=False)
"""
extrude_config = {
    'wafer':        gmshLayer_info('wafer', -d_wafer, d_wafer, 'DIELECTRIC'),
    'trap':         gmshLayer_info('bottom', 0, d_metal1, 'METAL'),
    'dielectric':   gmshLayer_info('gnd', 0, d_diel, 'DIELECTRIC', ('trap',)),
    'top':          gmshLayer_info('gnd', d_diel, d_metal2, 'METAL'),
    'helium':       gmshLayer_info('wafer', 0, d_He, 'HELIUM', ('trap', 'dielectric','top')),
    'vacuum':       gmshLayer_info('wafer', d_He, d_vac, 'VACUUM', ('dielectric', 'top'))
}

""" 
electrodes_config: 
key - physicalSurface name
item - physSurface_info dict:
       (geometry layer, electrode idx in geometry layer, gmsh layer)
"""
electrodes_config = {
    'mid':   physSurface_info('bottom', [0], 'trap'),
    'out':   physSurface_info('bottom', [1,2], 'trap'),
    'top':   physSurface_info('gnd', [0,1], 'top'),
}

scale = 4 # scaling factor for meshing. default 1.8

mesh_cfg = [{"Thickness": 4, "VIn": scale * 0.2, "VOut": 2, "box": [-10, 10, 10, 10, -5, 5]},
            {"Thickness": 2, "VIn": scale * 0.1, "VOut": 2, "box": [-8, 8, -8, 8, -2, 2]}
            ]

In [ ]:
meshMKR = GMSHmaker(layout=device, 
                     extrude_config=extrude_config, 
                     electrodes_config=electrodes_config, 
                     mesh_params=mesh_cfg,
                     filename="dot")
meshMKR.disable_consoleOutput()
# meshMKR.create_geo()          # Uncomment to create .geo file
meshMKR.create_mesh(dim=3)
# meshMKR.open_gmsh()
meshMKR.finalize()

Below is an example of created geometry and a mesh using GMSH.

<img src="images/fig1.png" width="800"/>

### **Helium Surface Displacement calculations**

In [ ]:
s = Structure()
s.add_layer('etch', device.etch)
s.quickplot(show_line_idx=True)

# We want to make boundary with line id 1 and 11 to be open boundary (Neumann boundary),
# at all other boundaries displacement is set to 0 (Dirichlet boundary)

In [ ]:
from zeroheliumkit.fem.heliumsurface import GMSHmaker2D, HeliumSurfaceFreeFEM

In [ ]:
gmshmkr = GMSHmaker2D(layout = s,
                      electode_config = {"type": "polygon",
                                         "bound": {"layer": ("etch", (0,)), "exclude": [1,11], "value": 0}},
                      mesh_config = [{"Thickness": 5, "VIn": 0.9, "VOut": 0.9, "box": [-20, -10, 20, 10]}],
                      filename = "heliumsurface",
                      savedir = save_dir)
gmshmkr.disable_consoleOutput()
gmshmkr.create_mesh()
#gmshmkr.open_gmsh()
gmshmkr.finalize()

In [ ]:
hsfreefem = HeliumSurfaceFreeFEM(fem_config=gmshmkr.export_physical())
res = hsfreefem.run_pyfreefem()

# plotting results of helium surface displacement for a bulk helium distance of 10 cm from the gnd surface
# note: the geometry units are in micrometers, so the bulk helium distance is set to 0.01 (10 cm)
hsfreefem.plot_results(res, bulk_helium_distance=0.01)

In [ ]:
curvature_config = hsfreefem.get_code_config(bulk_helium_distances=[0, 0.001, 0.01, 0.1], surface_helium_level=d_He)
# curvature_config

### **Creating FreeFem configuration**

In [ ]:
from zeroheliumkit.fem.freefemer import FFconfigurator, ExtractConfig, FreeFEM

In [ ]:
He_level = d_He

var_eps = {
        'DIELECTRIC': 11.0,
        'METAL': 1.0,
        'HELIUM': 1.057,
        'VACUUM': 1.0,
}

ffc = FFconfigurator(config_file="config/dot.yaml",
                     dielectric_constants=var_eps,
                     ff_polynomial=2,
                     extract_opt=[ExtractConfig("result1", 'phi', 'xy', (-10,10,201), (-10,10,201), d_He),
                                  ExtractConfig("result2", 'phi', 'xy', (-10,10,201), (-10,10,201), curvature_config),
                                  ExtractConfig("result3", 'Ez', 'yz', (-10,10,101), (-2,2,101), 0.0)
                                  ],
                     msh_refinements=None
                     )

In [ ]:
pyff = FreeFEM(config_file="config/dot.yaml")

In [ ]:
await pyff.run(cores=4, print_log=False, freefem_path=":/Applications/FreeFem++.app/Contents/ff-4.15.1/bin", remove_txt_files=True)

### **Parsing FEM results**

In [ ]:
# from zeroheliumkit.fem import *

In [ ]:
# result = read_ff_output(["dump/ff_data_dot_result1.parquet",
#                   "dump/ff_data_dot_result2.parquet",
#                   "dump/ff_data_dot_result3.parquet"],
#                  "dump/ff_data_dot_header.yaml")